In [410]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd

In [411]:
df = pd.read_csv("../data/housing_train_original.csv")

In [412]:
df = df.drop(["Адрес", "e-mail", "Телефон", "Объявление", "Агент", "Агентство"], axis=1)

In [413]:
for i in range(len(df)):
    temp = df["Комнаты"][i].split()
    if len(temp) > 1:
        df.loc[i, "Комнаты"] = temp[0]

for i in range(len(df)):
    temp = str(df["Этаж/Дом"][i]).split('/')
    if len(temp) > 1:
        df.loc[i, "Этаж/Дом"] = int(temp[1])

In [414]:
for column in df.columns:
    if str(df[column].dtype) == 'object':
        df[column] = pd.factorize(df[column])[0]

In [415]:
1 - df.isna().sum() / len(df)

Комнаты          1.000000
Район            1.000000
Этаж/Дом         1.000000
Общая площадь    1.000000
Жилая площадь    1.000000
Площадь кухни    1.000000
Цена             0.769231
dtype: float64

In [416]:
df = df.dropna()

In [417]:
x = df.drop(["Цена"], axis=1)
y = df["Цена"]

random_state = 64
# Разделяем данные на обучающий и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=random_state)

In [448]:
model = xgb.XGBRFRegressor(random_state=random_state,
                           n_estimators=13,
                           max_depth=7
                           )
model.fit(x_train, y_train)
print(1)

1


In [449]:
title = ["train", "test"]
for i, data in enumerate([[x_train, y_train], [x_test, y_test]]):
    prediction, target_list = model.predict(data[0]), data[1]
    mse = mean_squared_error(target_list, prediction)
    mape = mean_absolute_percentage_error(target_list, prediction)
    print(title[i], mse, mape)

train 92828.0460742715 0.08356153583231658
test 307902.97391697083 0.10767141869445161


In [421]:
df = pd.read_csv("../data/housing_train_original.csv")

df = df.drop(["Адрес", "e-mail", "Телефон", "Объявление", "Агент", "Агентство"], axis=1)

for i in range(len(df)):
    temp = df["Комнаты"][i].split()
    if len(temp) > 1:
        df.loc[i, "Комнаты"] = temp[0]

for i in range(len(df)):
    temp = str(df["Этаж/Дом"][i]).split('/')
    if len(temp) > 1:
        df.loc[i, "Этаж/Дом"] = int(temp[1])

for column in df.columns:
    if str(df[column].dtype) == 'object':
        df[column] = pd.factorize(df[column])[0]
        
for column in df.columns:
    if column == 'Цена':
        continue

    df[column] = df[column].fillna(round(df[column].mode()[0]))

df = df[df["Цена"].isna()]

x = df.drop(["Цена"], axis=1)
y = df["Цена"]

prediction = model.predict(x)
prediction_df = pd.DataFrame(prediction, columns=["Цена"])
prediction_df.to_csv("prediction.csv", index=False)